## Combine survey points 

In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd

In [2]:
points_20220103 = 'survey_points_20220103_revised20220121.shp'
points_20211213 = 'survey_points_20211213_quick_look_revised20220119.shp'
apple1 = 'survey_points_20210716.shp'
apple2 = 'survey_points_20210825.shp'

In [3]:
gt_dir = r'K:\2021-data-org\4. RESEARCH_n\ML\MLsatellite\Data\layers_india\GT\survey-farmer-locations\processed\shp'

In [72]:
points_20220103_shp = gpd.read_file(gt_dir + '/' + points_20220103)
points_20220103_shp.shape

In [74]:
points_20220103_shp.head()

,Date of th,Name of th,Contact nu,Gender,Are you a,If Yes_ Pl,If Yes_ _1,Site GPS l,Site GPS_1,Altitude (,...,Please m_1,Do you use,"If Yes, do","If Yes, 10",What is _5,Operating,assessment,check,comment,geometry
0,2021-05-20,Akhil Jinta,yb1t669yyy,Male,No,None,None,31.1922965118397,77.7477854408256,5800.0,...,"English, Hindi",Yes,Yes,Yes,Samsung,Android,NaN,None,None,POINT (77.74779 31.19230)
1,2021-06-21,Amit Jaita,xa16r3003r,Male,No,Progressive Growers Association,240.0,31.1154025094707,77.8020973736234,7300.0,...,"English, Hindi",Yes,Yes,Yes,Xiaomi,Android,NaN,None,None,POINT (77.80210 31.11540)
2,2021-05-22,P79kaj,91290z000r,Male,Yes,Baghi Apple Growers Society,None,31.1409520521778,77.7861961282442,6200.0,...,"English, Hindi",Yes,Yes,Yes,Vivo,Android,NaN,None,None,POINT (77.78620 31.14095)
3,2021-05-25,Raghav T79ta,yb1ttty6222,Male,Yes,Jubbal Apple Growers Society,300.0,31.1061073409128,77.6496921409057,7500.0,...,"English, Hindi",Yes,Yes,Yes,Apple,IoS,NaN,None,None,POINT (77.64969 31.10611)
4,2021-06-26,Zhrik79t,r01t32y9y3,Male,Yes,None,None,None,None,6000.0,...,English and Hindi,Yes,Yes,Yes,Xiaomi,Android,NaN,None,None,None


In [78]:
points_20211213_shp = gpd.read_file(gt_dir + '/' + points_20211213)
points_20211213_shp.shape

(483, 114)

In [ ]:
points_20211213_shp.head()

In [245]:
combined = points_20220103_shp.copy()
for r, row in points_20220103_shp[points_20220103_shp['assessment'].isnull()].iterrows():
    if row['geometry']:
        mask1 = points_20211213_shp['Site GPS l'].astype(float) == row['geometry'].y
        mask2 = points_20211213_shp['Site GPS_1'].astype(float) == row['geometry'].x
        corr_row = points_20211213_shp[mask1 & mask2]
        if corr_row.shape[0]:
            if corr_row.shape[0] != 1:
                print(f'Found {corr_row.shape[0]} matches...')
                print(row['geometry'].y, row['geometry'].x)
            assess = corr_row.loc[corr_row.index[0], 'assessment']
            check = corr_row.loc[corr_row.index[0], 'check']
            if not pd.isna(assess) and not pd.isna(check):
                combined.loc[r, 'assessment'] = assess
                combined.loc[r, 'check'] = check
            else: 
                combined.loc[r, 'assessment'] = 1
                combined.loc[r, 'check'] = 1
        else:
            combined.loc[r, 'assessment'] = -2
            combined.loc[r, 'check'] = -2
    else:
        combined.loc[r, 'assessment'] = None
        combined.loc[r, 'check'] = None

Found 2 matches...
32.098946 77.135877
Found 2 matches...
32.098946 77.135877


In [258]:
combined.assessment = combined.assessment.astype('Int64')
combined.check = pd.to_numeric(combined.check, downcast='integer').astype('Int64')

In [259]:
combined.dtypes

Date of th      object
Name of th      object
Contact nu      object
Gender          object
Are you a       object
                ...   
Operating       object
assessment       Int64
check            Int64
comment         object
geometry      geometry
Length: 114, dtype: object

In [262]:
combined.groupby('check', dropna=False).count()

,Date of th,Name of th,Contact nu,Gender,Are you a,If Yes_ Pl,If Yes_ _1,Site GPS l,Site GPS_1,Altitude (,...,Can you wr,Please m_1,Do you use,"If Yes, do","If Yes, 10",What is _5,Operating,assessment,comment,geometry
check,,,,,,,,,,,,,,,,,,,,,
0,28,28,27,28,28,20,19,28,28,18,...,28,28,28,28,28,25,25,28,2,28
1,360,360,340,356,359,304,275,360,360,123,...,360,360,360,360,360,335,333,360,0,360
2,90,90,85,90,90,85,76,90,90,22,...,90,90,90,90,90,79,79,90,0,90
NaN,1,1,1,1,1,0,0,0,0,1,...,1,1,1,1,1,1,1,0,0,0


In [260]:
combined.groupby('assessment', dropna=False).count()

,Date of th,Name of th,Contact nu,Gender,Are you a,If Yes_ Pl,If Yes_ _1,Site GPS l,Site GPS_1,Altitude (,...,Can you wr,Please m_1,Do you use,"If Yes, do","If Yes, 10",What is _5,Operating,check,comment,geometry
assessment,,,,,,,,,,,,,,,,,,,,,
-1,30,30,29,30,30,26,25,30,30,11,...,30,30,30,30,30,30,30,30,0,30
0,9,9,9,9,9,8,8,9,9,5,...,9,9,9,9,9,6,6,9,0,9
1,284,284,268,281,284,234,210,284,284,103,...,284,284,284,284,284,265,263,284,1,284
2,155,155,146,154,154,141,127,155,155,44,...,155,155,155,155,155,138,138,155,1,155
NaN,1,1,1,1,1,0,0,0,0,1,...,1,1,1,1,1,1,1,0,0,0


In [263]:
combined.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [265]:
combined.to_file(gt_dir + '/survey_points_combined_20210716_20210825_20211213_20220103.shp')

## Combine survey polygons

In [67]:
polygon_dir = r'K:\2021-data-org\4. RESEARCH_n\ML\MLsatellite\Data\layers_india\GT\survey-polygons'
apple_polygon1 = '/survey20210716_polygons20210819_corrected20210831.shp'
apple_polygon2 = '/survey20210825_polygons20210901_revised20210929.shp'
apple_polygon3 = '/survey20211213_polygons20220121.shp'
apple_polygon4 = '/survey20220103_polygons20220121.shp'

In [68]:
import geopandas as gpd

In [69]:
apple_polygon_shp1 = gpd.read_file(polygon_dir + apple_polygon1)
apple_polygon_shp2 = gpd.read_file(polygon_dir + apple_polygon2)
apple_polygon_shp3 = gpd.read_file(polygon_dir + apple_polygon3)
apple_polygon_shp4 = gpd.read_file(polygon_dir + apple_polygon4)

if not(apple_polygon_shp1.crs == apple_polygon_shp2.crs == apple_polygon_shp3.crs == apple_polygon_shp4.crs):
    apple_polygon_shp2 = apple_polygon_shp2.to_crs(apple_polygon_shp1.crs)
    apple_polygon_shp3 = apple_polygon_shp3.to_crs(apple_polygon_shp1.crs)
    apple_polygon_shp4 = apple_polygon_shp4.to_crs(apple_polygon_shp1.crs)

apple_polygons = gpd.GeoDataFrame(pd.concat([apple_polygon_shp1, apple_polygon_shp2, apple_polygon_shp3, apple_polygon_shp4], ignore_index=True))
apple_polygons = apple_polygons.set_crs(apple_polygon_shp1.crs)
apple_polygons.to_file(polygon_dir + '/combined/polygons_surveys_20210716_20210825_20211213_20220103.shp')

## Combine cropland polygons

In [285]:
gt_dir = r'K:\2021-data-org\4. RESEARCH_n\ML\MLsatellite\Data\layers_india\GT'
apple_polygon1 = '/survey20210716_polygons20210819_corrected20210831.shp'
apple_polygon2 = '/survey20210825_polygons20210901_revised20210929.shp'

k_drive = r'K:\2021-data-org\4. RESEARCH_n\ML\MLsatellite\Research\WP1_Danya\ML4Satellite\data\ground_truth'
non_crops_polygon = '/non_crops/non_crops.shp'
other_crops_polygon = '/other_crops/other_crops.shp'
test_polygons_near = '/test_polygons_near/test_polygons_near.shp'
test_polygons_far = '/test_polygons_far/test_polygons_far.shp'

polygon_dir = r'K:\2021-data-org\4. RESEARCH_n\ML\MLsatellite\Data\layers_india\GT\survey-polygons'

In [279]:
apple_polygon_shp1 = gpd.read_file(polygon_dir + apple_polygon1)
apple_polygon_shp2 = gpd.read_file(polygon_dir + apple_polygon2)

In [284]:
non_crops_shp = gpd.read_file(k_drive + non_crops_polygon)
other_crops_shp = gpd.read_file(k_drive + other_crops_polygon)
test_near_shp = gpd.read_file(k_drive + test_polygons_near)
test_far_shp = gpd.read_file(k_drive + test_polygons_far)

In [292]:
if not (apple_polygon_shp1.crs == apple_polygon_shp2.crs == non_crops_shp.crs == other_crops_shp.crs == test_far_shp.crs == test_near_shp.crs):
    apple_polygon_shp2 = apple_polygon_shp2.to_crs(apple_polygon_shp1.crs)
    non_crops_shp = non_crops_shp.to_crs(apple_polygon_shp1.crs)
    other_crops_shp = other_crops_shp.to_crs(apple_polygon_shp1.crs)
    test_near_shp = test_near_shp.to_crs(apple_polygon_shp1.crs)
    test_far_shp = test_far_shp.to_crs(apple_polygon_shp1.crs)

train_shp = gpd.GeoDataFrame(pd.concat([apple_polygon_shp1[apple_polygon_shp1.district == 'Kullu'], 
                                        apple_polygon_shp2[apple_polygon_shp2.district == 'Kullu'], 
                                        non_crops_shp, other_crops_shp], ignore_index=True))
test_shp = gpd.GeoDataFrame(pd.concat([apple_polygon_shp1[apple_polygon_shp1.district != 'Kullu'], 
                                       apple_polygon_shp2[apple_polygon_shp2.district != 'Kullu'], 
                                       test_near_shp, test_far_shp], ignore_index=True))
 
train_shp = train_shp.set_crs(apple_polygon_shp1.crs)
test_shp = test_shp.set_crs(apple_polygon_shp1.crs)

train_shp.loc[train_shp.id == 1, 'id'] = 2
test_shp.loc[test_shp.id == 1, 'id'] = 2

In [293]:
train_shp

,id,district,geometry
0,2,Kullu,"POLYGON ((77.12878 32.05505, 77.12891 32.05526..."
1,2,Kullu,"POLYGON ((77.12814 32.05329, 77.12858 32.05315..."
2,2,Kullu,"POLYGON ((77.12329 32.09353, 77.12294 32.09258..."
3,2,Kullu,"POLYGON ((77.12360 32.09328, 77.12374 32.09321..."
4,2,Kullu,"POLYGON ((77.12452 32.09245, 77.12481 32.09236..."
...,...,...,...
215,2,Kullu,"POLYGON ((77.06560 32.10466, 77.06436 32.10564..."
216,2,Kullu,"POLYGON ((77.07429 32.10560, 77.07514 32.10558..."
217,2,Kullu,"POLYGON ((77.04770 32.12151, 77.04883 32.12285..."
218,2,Kullu,"POLYGON ((77.05132 32.11779, 77.05169 32.11870..."


In [294]:
train_shp.to_file(gt_dir + '/cropland-polygons/train_polygons.shp')
test_shp.to_file(gt_dir + '/cropland-polygons/test_polygons.shp')

## Split train and test polygons for apple classification

In [1]:
import geopandas as gpd

In [70]:
apple_dir = r'K:\2021-data-org\4. RESEARCH_n\ML\MLsatellite\Research\WP1_Danya\ML4Satellite\data\ground_truth\apple_polygons'
apple_name = '/polygons_surveys_20210716_20210825_20211213_20220103.shp'

In [71]:
apple_shp = gpd.read_file(apple_dir + apple_name)

In [72]:
apple_shp[apple_shp.district == 'Kullu'].to_file(apple_dir + '/kullu.shp')
apple_shp[apple_shp.district == 'Shimla'].to_file(apple_dir + '/shimla.shp')
apple_shp[apple_shp.district == 'Mandi'].to_file(apple_dir + '/mandi.shp')

In [22]:
img_path = r'N:\dataorg-datasets\MLsatellite\sentinel2_images\images_danya\43SFR\raster_sample\L2A_T43SFR_A014860_20200110T053549.tiff'
with rasterio.open(img_path, 'r') as f:
    meta = f.meta

In [8]:
kullu_apple = apple_shp[apple_shp.district == 'Kullu']
mandi_apple = apple_shp[apple_shp.district == 'Mandi']
shimla_apple = apple_shp[apple_shp.district == 'Shimla']

In [12]:
from rasterio.features import rasterize

In [43]:
def shp_to_arr(shp, transform):
    minx, miny, maxx, maxy = shp.to_crs(meta['crs']).total_bounds
    row_min, col_min = rasterio.transform.rowcol(meta['transform'], minx, maxy)
    row_max, col_max = rasterio.transform.rowcol(meta['transform'], maxx, miny)
    
    val_list = list(shp.id.values)
    features_list = [shapely.geometry.mapping(s) for s in shp.geometry]
    iterable = iter([(feature, val) for feature, val in zip(features_list, val_list)])
    img = rasterize(iterable, out_shape=(row_max - row_min, col_max - col_min), transform=transform)

    return img

In [44]:
kullu_raster = shp_to_arr(kullu_apple, meta['transform'])
mandi_raster = shp_to_arr(mandi_apple, meta['transform'])
shimla_raster = shp_to_arr(shimla_apple, meta['transform'])

In [45]:
(kullu_raster == 1).sum(), (mandi_raster == 1).sum(), (shimla_raster == 1).sum()

(0, 0, 0)

In [53]:
shp = kullu_apple.copy()
shp = shp.to_crs(meta['crs'])
minx, miny, maxx, maxy = shp.total_bounds
row_min, col_min = rasterio.transform.rowcol(meta['transform'], minx, maxy)
row_max, col_max = rasterio.transform.rowcol(meta['transform'], maxx, miny)

val_list = list(shp.id.values)
features_list = [shapely.geometry.mapping(s) for s in shp.geometry]
iterable = iter([(feature, val) for feature, val in zip(features_list, val_list)])
img = rasterize(iterable, out_shape=(row_max - row_min, col_max - col_min), transform=meta['transform'])

In [54]:
img.sum()

0

In [64]:
iterable = iter([(feature, val) for feature, val in zip(features_list, val_list)])

In [61]:
next(iterable)

({'type': 'Polygon',
  'coordinates': (((700975.0826736939, 3548519.2722491897),
    (700986.3411852395, 3548543.555025041),
    (701008.6905876274, 3548551.100427416),
    (701042.1787254864, 3548534.319222642),
    (701049.6890306589, 3548530.106930698),
    (701056.9526234203, 3548524.9553797003),
    (701066.3620302923, 3548518.6003254065),
    (701072.8486151878, 3548512.4990537125),
    (701085.9008063462, 3548482.856038777),
    (701045.0228856088, 3548296.1046916386),
    (700975.0905713942, 3548318.084774219),
    (700911.7099909534, 3548371.029514106),
    (700941.2116533124, 3548445.428489786),
    (700956.0182157005, 3548478.112884507),
    (700975.0826736939, 3548519.2722491897)),
   ((701004.1985537646, 3548429.2290306897),
    (700999.6090201657, 3548406.4015463577),
    (701024.4098462404, 3548399.7272007703),
    (701027.9079880094, 3548424.09048792),
    (701004.1985537646, 3548429.2290306897)),
   ((700985.5106717678, 3548381.5176806096),
    (700980.7357553793, 3548

In [65]:
rasterize(iterable, out_shape=(row_max - row_min, col_max - col_min)).sum()

0

In [73]:
n_kullu = 6743
n_mandi = 4422
n_shimla = 10761

In [75]:
n_tot = n_kullu + n_mandi + n_shimla
n_tot

21926

In [76]:
n_mandi / n_tot

0.2016783727082003